In [ ]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Pull the known results
df_fixtures = pd.read_csv("https://fixturedownload.com/download/fifa-world-cup-2022-UTC.csv")
df_results = df_fixtures[df_fixtures["Result"].notna()]

In [ ]:
# Split Result into home goals and away goals
df_results["Home Goals"] = df_results["Result"].apply(lambda x: int(x.split("-")[0]))
df_results["Away Goals"] = df_results["Result"].apply(lambda x: int(x.split("-")[1]))

In [ ]:
# Get the total goals for each team
df_goals = pd.concat(
    [
        df_results[[f"{x} Team", f"{x} Goals"]].rename(columns = lambda x: x.split(" ")[1]) 
        for x in ["Home", "Away"]
    ]
)
df_goals = df_goals.groupby("Team")[["Goals"]].sum().reset_index()

In [ ]:
# Read the players picks
df_players_picks = pd.read_csv("df_players_picks.csv")

In [ ]:
# Clean the players picks
df_players_picks = df_players_picks.set_index("name")
df_players_picks["picks"] = df_players_picks["picks"].apply(lambda x: x.replace("[", "").replace("]", "").replace("'", ""))

In [ ]:
# Convert the picks into seperate columns
df_players_goals = pd.DataFrame(df_players_picks["picks"].str.split(", ").tolist(), index=df_players_picks.index)

In [ ]:
# Convert to long
df_players_goals = df_players_goals.stack().to_frame("Team")
df_players_goals.index = df_players_goals.index.droplevel(1)

In [ ]:
# Join the goals and aggregate to each player
df_players_goals = pd.merge(df_players_goals.reset_index(), df_goals, on="Team", how="left").fillna(0)
df_players_goals = df_players_goals.groupby("name")[["Goals"]].sum().astype(int)

In [ ]:
# Join the total goals the the original dataframe
df_players_picks_with_goals = df_players_picks.join(df_players_goals)

In [ ]:
df_players_picks_with_goals = df_players_picks_with_goals.sort_values("Goals", ascending=False)

In [ ]:
# Colour the table based on top score or bust
top_score = df_players_picks_with_goals["Goals"].apply(lambda x: 0 if x > 21 else x).max()

def colour(x):
    if x > 21:
        c = "red"
    elif x == top_score:
        c = "green"
    else:
        return None
    return f"background-color: {c}"

df_players_picks_with_goals.style.applymap(colour, subset=["Goals"])

In [ ]:
print(datetime.now())